## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Souillac,MU,-20.5167,57.5167,78.8,94,75,13.80,light rain,2021-01-11 04:37:33
1,1,Tasiilaq,GL,65.6145,-37.6368,19.4,63,90,3.44,overcast clouds,2021-01-11 04:37:42
2,2,Mar Del Plata,AR,-38.0023,-57.5575,64.4,94,90,6.91,heavy intensity rain,2021-01-11 04:37:46
3,3,East London,ZA,-33.0153,27.9116,66.2,94,90,2.30,light rain,2021-01-11 04:35:31
4,4,Fortuna,US,40.5982,-124.1573,50.0,100,1,3.65,clear sky,2021-01-11 04:38:49


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Souillac,MU,-20.5167,57.5167,78.80,94,75,13.80,light rain,2021-01-11 04:37:33
8,8,Rikitea,PF,-23.1203,-134.9692,78.24,79,35,18.03,scattered clouds,2021-01-11 04:35:20
13,13,Touros,BR,-5.1989,-35.4608,78.80,78,40,4.61,scattered clouds,2021-01-11 04:38:52
21,21,Clarence Town,AU,-32.5833,151.7833,84.99,54,20,16.11,few clouds,2021-01-11 04:38:53
22,22,Cidreira,BR,-30.1811,-50.2056,76.50,83,14,22.70,few clouds,2021-01-11 04:37:43
26,26,Bengkulu,ID,-3.8004,102.2655,85.14,64,100,6.64,light rain,2021-01-11 04:37:25
27,27,Avarua,CK,-21.2078,-159.7750,82.40,78,20,10.36,few clouds,2021-01-11 04:37:13
32,32,Domoni,KM,-12.2569,44.5319,81.52,79,98,9.53,overcast clouds,2021-01-11 04:38:55
34,34,Kanniyakumari,IN,8.0833,77.5667,80.51,75,96,18.30,heavy intensity rain,2021-01-11 04:38:56
38,38,Byron Bay,AU,-28.6500,153.6167,77.00,73,40,16.11,scattered clouds,2021-01-11 04:38:57


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                173
City                   173
Country                172
Lat                    173
Lng                    173
Max Temp               173
Humidity               173
Cloudiness             173
Wind Speed             173
Current Description    173
Date                   173
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Souillac,MU,78.80,light rain,-20.5167,57.5167,
8,Rikitea,PF,78.24,scattered clouds,-23.1203,-134.9692,
13,Touros,BR,78.80,scattered clouds,-5.1989,-35.4608,
21,Clarence Town,AU,84.99,few clouds,-32.5833,151.7833,
22,Cidreira,BR,76.50,few clouds,-30.1811,-50.2056,
26,Bengkulu,ID,85.14,light rain,-3.8004,102.2655,
27,Avarua,CK,82.40,few clouds,-21.2078,-159.7750,
32,Domoni,KM,81.52,overcast clouds,-12.2569,44.5319,
34,Kanniyakumari,IN,80.51,heavy intensity rain,8.0833,77.5667,
38,Byron Bay,AU,77.00,scattered clouds,-28.6500,153.6167,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
print(hotel_df.head(10))

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
             City Country  Max Temp   Current Description      Lat       Lng  \
0        Souillac      MU     78.80            light rain -20.5167   57.5167   
8         Rikitea      PF     78.24      scattered clouds -23.1203 -134.9692   
13         Touros      BR     78.80      scattered clouds  -5.1989  -35.4608   
21  Clarence Town      AU     84.99            few clouds -32.5833  151.7833   
22       Cidreira      BR     76.50            few clouds -30.1811  -50.2056   
26       Bengkulu      ID     85.14            light rain  -3.8004  102.2655   
27         Avarua      CK     82.40            few clouds -21.2078 -159.7750   
32         Domoni      KM     81.52       overcast clouds -12.2569   44.5319   
34  Kanniyakumari      IN     80.51  heavy intensity rain   8.0833   77.5667   
38      Byron Bay      AU 

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Souillac,MU,78.80,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
8,Rikitea,PF,78.24,scattered clouds,-23.1203,-134.9692,Pension Maro'i
13,Touros,BR,78.80,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
21,Clarence Town,AU,84.99,few clouds,-32.5833,151.7833,River Downs Cottage
22,Cidreira,BR,76.50,few clouds,-30.1811,-50.2056,Hotel Castelo
...,...,...,...,...,...,...,...
674,Ambanja,MG,77.00,light rain,-13.6833,48.4500,Hotel Salama and Fast Food
677,Bhadrachalam,IN,84.63,broken clouds,17.6667,80.8833,Hotel Srinidhi Residency
681,Galesong,ID,86.00,moderate rain,-5.3166,119.3661,Sapriadi Appy
684,Calabozo,VE,75.33,few clouds,8.9242,-67.4293,Hotel Giardini


In [14]:
# 8a. Create the output File (CSV)
output_data_file ="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name </dt><dd>{Hotel Name}</dd>
<dt>City </dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))